## Pre-processing :EMA and sensing data
    - slice
    - merge
    - interpolate

In [1]:
import numpy as np
import pandas as pd
import random

dir_data ='../data/processed_data/'
dir_code ='./'

## process EMA data
    - Original data in json file
    - step 1 to slice json file to csv file by folders, slice by 24 hours, average activities
    - step 2 is to concatenate all csv files into ema_all_v5.csv

In [5]:
import pandas as pd
import re
import os
from util import stmp_to_fmt, fmt_to_stmp

# dir_data='D:/research/dataset/studentlife/EMA/response/' # lab win10
dir_data= 'C:/study/studentlife/EMA/response/' #home win10

dir_code = './'
dir_tosave = '../data/processed_data'

In [6]:
################ main function of average the acitivities/value by days#################
## concatenate all candicates' acitibity/day by type of activity #########
import numpy as np
def count_ema_response(file_dir, folder_name): # count all columns exept ['location','resp_time']
    count=0
    dataset = pd.DataFrame() # creat a empty dataframe
    file_list= sorted(os.listdir(file_dir))
#     file_list=['Activity_u00.json']

    for file in file_list:

        user = re.findall('u[0-9][0-9]', file)[0]
        data = pd.read_json(file_dir+file, convert_dates= False) # keep timestamp formate
        data.fillna(0, inplace =True)
        #print(file,' data sieze: ',data.shape)
        if data.empty:
            continue # if empty go to next file, note; use break will jump out of def
        
        try:
            data= data.drop('location', axis =1) # filter location column
        except: 
            pass
        
        len=data.shape[0]
        if np.any(data.isnull()):
            data=data.dropna(axis=0) # filter null data
        data=data.set_index('resp_time')
        slice_width = 1*3600*24
        for time in range(int(data.index[0]), int(data.index[-1])+1,slice_width):  # timestamp is index, +1 is for data with only one sample
            #data_slice = data.loc[time:time+slice_width-1,:]  # data.loc: slice of both left and right boundary is included, so -1
            slice_index= data.index.where((data.index>= time) & (data.index< time+slice_width)).dropna()
            data_slice =data.loc[slice_index,:]
#             acts=(data_slice.sum(axis =0))  # count the activities in the time slot
            acts=(data_slice.mean(axis =0,skipna=True, numeric_only= True))  # AVERAGE the activities in the time slot
            acts['user']=user
            #acts['timestamp']=time 
#             print(type(user))
            acts['user_date']=user+'_'+stmp_to_fmt(time)[:10] # pickup the yearMMDD
            acts=acts.to_frame().T
 
            dataset=pd.concat([dataset, acts], axis =0)  #append the user infor
            #end of one file processed
        
        count+=1 # end of processing a file
        # end of one folder processed
    
    dataset=dataset.set_index('user_date')
    col=dataset.columns
    new_col=folder_name+'_'+col
    dataset.columns = new_col
    print("Total ",count, " files are processed, in the fold: ", folder_name )
    return dataset

In [7]:
##### A. generate dataset by types for all folders #######
import os

ii=0
folder_list= os.listdir(dir_data)
for folder_name in folder_list: #'activity. behaviour, class ...'
    file_dir = dir_data+folder_name+'/' # e,g, 'C:/study/studentlife/EMA/response/Activity'
    data=count_ema_response(file_dir, folder_name)
    data.to_csv(dir_tosave+folder_name+'.csv')
    ii +=1

print('Total ', ii, ' folders', 'process!')
print("Done! Files saved in  : ", dir_tosave)

In [8]:
########### B. concat all data in file ############
ema_files_position = dir_tosave
files = os.listdir(ema_files_position)
# files = ['Activity.csv','Class.csv'] # for testing
data_all = pd.DataFrame()
for file in files:
    this_data= pd.read_csv(ema_files_position+file, index_col=0)
    data_all= data_all.merge(this_data, how='outer', left_index=True, right_index=True )
print('Merge success ! Data size of merged dataset is', data_all.shape )
dir_tosave_ema = '../data/processed_data/' # home win10
data_all.to_csv(dir_tosave_ema+ 'ema_all_v5.csv')

In [ ]:
########## 0. activity ###################
# folder_name='Activity'
# file_dir = dir_data+folder_name+'/'
# count_ema_response(file_dir, folder_name)

## process sensing data
    - slice, sum/calculate duration, concatenate
    - process 'activity' and 'audeo' by sum
    - process 'conversation','dark','phonecharge','phonelock' by calculating diration

In [9]:
import pandas as pd
import re
import os
import util
import numpy as np
from util import stmp_to_fmt, fmt_to_stmp


dir_data= 'C:/study/studentlife/sensing/' #home 
dir_code = './'
dir_tosave = '../data/processed_data/sensing/' 


In [10]:
##### 1. take a look at the whole data info in "sensing" folder #######
import os

def get_file_profile(sub_folder_dir,  file_name):
    dataset = pd.DataFrame() # creat a empty dataframe
    user = re.findall('u[0-9][0-9]', file_name)
    data = pd.read_csv(sub_folder_dir+file_name)
    col=list(data.columns)
    time = data.iloc[:,0]
    data_size = data.shape[0]
    days = int((data.iloc[-1,0]-data.iloc[0,0])/3600/24)

#     data_info_all =pd.DataFrame(columns =['file name','userID','columns','start_time','end_time','duration(days)','instance_no'])
    data_info= [file_name]+user+[','.join(col[:])]+[stmp_to_fmt(int(data.iloc[0,0]))]+[stmp_to_fmt(int(data.iloc[-1,0]))]+[days]+[data_size]
    return data_info

sub_folder_list =[]
folder_list= os.listdir(dir_data)

i=0
data_info_all =pd.DataFrame(columns =['file name','userID','columns','start_time','end_time','duration(days)','instance_no'])
for sub_folder_name in folder_list:  # folder of sensing
    sub_folder_list = sub_folder_list+[sub_folder_name]
    sub_folder_dir = dir_data+sub_folder_name+'/'
    file_list = os.listdir(sub_folder_dir)
    for file_name in file_list:
        data_info= get_file_profile(sub_folder_dir,  file_name)
        a_series = pd.Series(data_info, index = data_info_all.columns)
        data_info_all=data_info_all.append(a_series, ignore_index=True)
        i+=1
#         print(file_name)

print('total files: ',i)
print(sub_folder_list)
# data_info_all.to_csv(dir_data+"sensing_data_info_summary.csv")
data_info_all

In [11]:
########## 1.2  create a data dictionary for slicing ###########
# start_date = '2013-03-26 00:00:00'
# end_date ='2013-05-31 23:59:59'
# start_date_stmp= fmt_to_stmp(start_date,"%Y-%m-%d %H:%M:%S")
# end_date_stmp= fmt_to_stmp(end_date,"%Y-%m-%d %H:%M:%S")
# # print(start_date_stmp, ' to ',end_date_stmp)

# date_dict={}
# i=0
# for time_stmp in range(start_date_stmp, end_date_stmp, 3600*24):
#     date=stmp_to_fmt(time_stmp)
#     i+=1
#     date_dict[date]=time_stmp, time_stmp+24*3600-1

In [12]:
##########  2.1. process(slice) activity's and audio's: sum activities ######
########### not include wifi , bluetooth, gps. wifi_location#########

def act_count_sliced(dir,file):
    user = re.findall('u[0-9][0-9]', file)[0]
    file= dir+file
    data= pd.read_csv(file, index_col= 0) # index_col= 0: use the first column as the index
#     data.fillna(0, inplace =True)  # if there is null in data set , add this
    if np.any(data.isnull()):
        data=data.dropna(axis=0) # filter null data
    
    slice_width = 1*3600*24
#     time_list=[]
    dataset=pd.DataFrame()
    for time in range(data.index[0], data.index[-1]+1,slice_width):  # timestamp is index
    #     time_list.append(time)
        data_slot = data.loc[time:time+slice_width-1,:]  #note: data.loc: slice of both left and right boundary is included, so -1
        acts=(data_slot.sum(axis =0))  # count the activities in the time slot
        
        #acts['timestamp']=time 
        acts['user']=user
        acts['user_date']=user+'_'+stmp_to_fmt(time)[:10]
        acts=acts.to_frame().T
        dataset=pd.concat([dataset, acts], axis =0)  #append the user infor
        #end of one file processed
        
    dataset=dataset.set_index('user_date')
    dataset=dataset.fillna(0)
#     print(acts)
    return dataset


# dataset =pd.DataFrame()
# dir_temp=(dir_data+'activity/')
# file= 'activity_u00.csv'
# data_s = act_count_sliced(dir_temp,file)
# dataset = pd.concat([dataset,data_s], axis =0) 
# dataset

dataset =pd.DataFrame()
file_dir = dir_data+'activity/'
file_list= sorted(os.listdir(file_dir))
print("total ", len(file_list)," files in the folder of " , file_dir)
print("counting........")
for file in file_list:
    data_= act_count_sliced(file_dir, file)
    dataset = pd.concat([dataset,data_], axis =0) 
col=dataset.columns
new_col=folder+'_'+col
dataset.columns = new_col
print("done!")
dataset.to_csv(dir_tosave+'activity.csv')

dataset =pd.DataFrame()
file_dir = dir_data+'audio/'
file_list= sorted(os.listdir(file_dir))
print("total ", len(file_list)," files in the folder of " , file_dir)
print("counting........")
for file in file_list:
    data_= act_count_sliced(file_dir, file)
    dataset = pd.concat([dataset,data_], axis =0) 
col=dataset.columns
new_col=folder+'_'+col
dataset.columns = new_col
print("done!")
dataset.to_csv(dir_tosave+'audio.csv')

In [13]:
###########  2.2 process (slice) 'conversation','dark','phonecharge','phonelock': calculate duration  ################
def duration_count_sliced(dir,file): #for col 0 is stat_time, col 1 is end_time
    user = re.findall('u[0-9][0-9]', file)[0]
    file= dir+file 
    data= pd.read_csv(file, index_col=0) # index_col= 0: use the first column as the index
    if np.any(data.isnull()):
        data=data.dropna(axis=0) # filter null data
    duration = data.iloc[:,0]-data.index # end_time - start_time
    data.insert(1,'duration(sec)',duration) # add column 
    slice_width = 1*3600*24

#     time_list=[]
    dataset=pd.DataFrame()
    for time in range(int(data.index[0]), int(data.index[-1])+1,slice_width):  # timestamp is index, +1 is for data with only one sample
#         data_slice = data.loc[time:time+slice_width-1,:]  # data.loc: slice of both left and right boundary is included, so -1
        slice_index= data.index.where((data.index>= time) & (data.index< time+slice_width)).dropna()
        data_slice =data.loc[slice_index,:]

        acts=(data_slice.iloc[:,1:].sum(axis =0))  # calculate the total duration in the sliced period, after column 1
        #acts['timestamp']=time 
        acts['user']=user
        acts['user_date']=user+'_'+stmp_to_fmt(time)[:10]        
        acts=acts.to_frame().T
    #     time_list.append(time)
        dataset=pd.concat([dataset, acts], axis =0)  #append the user infor
    dataset=dataset.set_index('user_date')
    dataset=dataset.fillna(0)
    return dataset

# dataset =pd.DataFrame()
# dir_temp=(dir_data+'phonecharge/')
# file= 'phonecharge_u39.csv'

# data_s = duration_count_sliced(dir_temp,file)
# dataset = pd.concat([dataset,data_s], axis =0) 
# dataset

folder_list= ['conversation','dark','phonecharge','phonelock']
# folder_list= ['phonelock']
for folder in folder_list:
    dataset =pd.DataFrame()
    file_dir = dir_data+folder+'/'
    file_list= sorted(os.listdir(file_dir))
    print("total ", len(file_list)," files in the folder of " , file_dir)
    print("counting........")
    for file in file_list:
#         print(file)
        data_= duration_count_sliced(file_dir, file)
        dataset = pd.concat([dataset,data_], axis =0) 
    col=dataset.columns
    new_col=folder+'_'+col
    dataset.columns = new_col
    print("done!")
    dataset.to_csv(dir_tosave+folder+'.csv')
    
##### seperately generate dataset by types for all folders, done ! #######

In [14]:
########### 3. concatenate all sensing data in one file ############

files_position = dir_tosave #'C:/study/Dropbox/research/depression/data/studentlife/sensing/'
files = os.listdir(files_position) #['Activity.csv','Class.csv', 'Class 2.csv',...]
# files = ['Activity.csv','Class.csv', 'Class 2.csv'] # for testing
data_all = pd.DataFrame()
for file in files:
#     print(file)
    this_data= pd.read_csv(files_position+file, index_col='user_date')
#     print(this_data)
#     data_all=pd.concat([data_all,this_data], axis=1,ignore_index=False)
    data_all= data_all.merge(this_data, how='outer', left_index=True, right_index=True )
print('Merge success ! Data size of merged dataset is', data_all.shape )
data_all
dir_tosave_sensing= 'C:/study/Dropbox/research/depression/code/data/' # home win10
data_all.to_csv(dir_tosave_sensing+ 'sensing_all_v2.csv')

## Merge EMA and sensing data

In [4]:
#################  1.1 merge sensing and ema  ####################
######### ema_all_v5.csv, sensing_all_v2.csv  ###################

dir_data = '../data/processed_data/'

dataset_ema = pd.read_csv(dir_data+ 'ema_all_v5.csv', index_col=0)  #v2 is the sliced dataset
print ('1. Dataset_ema shape: ',dataset_ema.shape)

dataset_sensing = pd.read_csv(dir_data+ 'sensing_all_v2.csv', index_col=0)  #v2 is the sliced dataset
print ('2. Dataset_sensing shape: ',dataset_sensing.shape)

######### clean data ###########
dataset= dataset_sensing.merge(dataset_ema, how='outer', left_index=True, right_index=True )
print('3. Merge success ! shape of merged dataset is', dataset.shape )

def drop_notnum(dataset):
    cols=dataset.columns
    col_notnum=[]
    for col in cols:
        if str(dataset[col].dtype)=='object':
               col_notnum.append(col)
    # print(col_notnum)
    dataset=dataset.drop(col_notnum, axis=1)
    return dataset

dataset=drop_notnum(dataset)  #abc=dataset.dropna(axis=1, how='all') # drops columns those with all NaN
dataset_ema= drop_notnum(dataset_ema)
dataset_sensing = drop_notnum(dataset_sensing)
print('')
print('4.1 Removed columns of non-data, merged dataset shape: ', dataset.shape)
print('4.2 Removed columns of non-data, EMA dataset shape: ', dataset_ema.shape)
print('4.2 Removed columns of non-data, sensing dataset shape: ', dataset_sensing.shape)

dataset.to_csv(dir_data+"studentlife_data_full.csv")

1. Dataset_ema shape:  (3025, 107)
2. Dataset_sensing shape:  (3075, 12)
3. Merge success ! shape of merged dataset is (3407, 119)

4.1 Removed columns of non-data, merged dataset shape:  (3407, 87)
4.2 Removed columns of non-data, EMA dataset shape:  (3025, 81)
4.2 Removed columns of non-data, sensing dataset shape:  (3075, 6)
